In [14]:
import os
from PIL import Image
import numpy as np

class ImageStatsCalculator:
    def __init__(self, folder_path):
        self.folder_path = folder_path
        self.mean_images, self.std_images = self.calculate_stats(os.path.join(folder_path, 'images'))
        self.mean_masks, self.std_masks = self.calculate_stats(os.path.join(folder_path, 'masks'))

    def calculate_stats(self, folder_path):
        image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
        image_count = len(image_files)
        channel_sums = np.zeros(3, dtype=np.float64)
        channel_sums_squared = np.zeros(3, dtype=np.float64)

        for image_file in image_files:
            image_path = os.path.join(folder_path, image_file)
            img = np.array(Image.open(image_path))

            if img.ndim == 2:  # Single-channel image
                img = np.expand_dims(img, axis=-1)  # Add an extra channel dimension

            # Flatten the image to calculate mean and std across all channels
            flattened_img = img.reshape(-1, img.shape[-1])

            channel_sums += np.sum(flattened_img, axis=0)
            channel_sums_squared += np.sum(flattened_img ** 2, axis=0)

        mean = channel_sums / (image_count * img.size)
        variance = ((channel_sums_squared / (image_count * img.size)) - mean ** 2)

        # Clip to ensure non-negative variance values
        std = np.sqrt(np.clip(variance, a_min=0, a_max=None))
        # std = variance


        return mean.tolist(), std.tolist()

if __name__ == "__main__":

    dataset="ISIC2018"
    folder_path = f"../../others_work/dataset/{dataset}/train_folder"
    stats_calculator = ImageStatsCalculator(folder_path)

    print("均值和方差（images文件夹）：")
    print("均值：", stats_calculator.mean_images)
    print("方差：", stats_calculator.std_images)

    print("均值和方差（masks文件夹）：")
    print("均值：", stats_calculator.mean_masks)
    print("方差：", stats_calculator.std_masks)


均值和方差（images文件夹）：
均值： [60.215605702690645, 49.47938440048857, 45.564065575875404]
方差： [0.0, 0.0, 0.0]
均值和方差（masks文件夹）：
均值： [54.57621928812819, 54.57621928812819, 54.57621928812819]
方差： [0.0, 0.0, 0.0]
